![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('max_rows', 60)
pd.set_option('max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

import re

In [120]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [121]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [122]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [123]:
df_train.info()

In [124]:
df_test.info()

In [125]:
sample_submission.info()

In [126]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [127]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [128]:
data.sample(10)

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

# Заполним пропуски в Number of Reviews средним значением

In [129]:
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [130]:
value = data['Number of Reviews'].mean()
data['Number of Reviews'].fillna(value, inplace=True)

# Заполним пропуски в Price Range средним значением

In [131]:
data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')

In [132]:
value = data['Price Range'].value_counts().index[0]
data['Price Range'].fillna(value, inplace=True)

In [133]:
data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')

In [134]:
data['Cuisine Style'].fillna("['Other']", inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [135]:
data.nunique(dropna=False)

In [136]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

#### Возьмем следующий признак "Price Range".

In [137]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [138]:
# строку в список
def to_list(data):
    pattern = re.compile(r'\[|\"|\'|\]')
    if isinstance(data, str):
        return re.sub(pattern, '', data).split(', ')
    else:
        return []


# приводим Prise Range к числовому формату
def transform_price_range(data):
    if data == '$':
        return 1
    elif data == '$$ - $$$':
        return 2
    else:
        return 0

### Признак Price Range, представленный в числовом виде.

In [139]:
data['Price_Range_Numb'] = data['Price Range'].apply(transform_price_range)

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Набор фиктивных признаков Price Range.

In [140]:
data = pd.get_dummies(data, columns=[ 'Price Range',])

### Обработка признака Cuisine Style.

In [141]:
def in_list_or_str(x, value):
    if isinstance(x, str):
        return value == x
    else:
        return value in x

# создать датафрейм, столбцы которого представлены уникальными значениями передаваемого столбца и
# содержат 1, если запись принадлежит к текущему столбцу, иначе 0
def to_dummy_var(column, new_columns=None):
    if not new_columns:
        new_columns = column.unique()
    store = []
    new_df = pd.DataFrame(columns=new_columns)
    for value in new_columns:
        series = column.apply(lambda x: 1 if in_list_or_str(x, value) else 0)
        new_df[f'Cuisine_Style_{value}'] = series
    return new_df

# количество каждой кухни
# функция используется для получения уникальных
def unique_cuisines(column):
    cuisines = set()
    for row in column:
        if row is not np.nan:
            cuisines = cuisines.union(set(row))
    return cuisines

### Количество типов кухни, по каждому ресторану.

In [142]:
data['Count_Cuisine_Style'] = data['Cuisine Style'].apply(to_list).apply(lambda x: len(x))

### Набор фиктивных признаков по признаку Cuisine Style.

In [143]:
styles = list(unique_cuisines(data['Cuisine Style'].apply(to_list)))
data = pd.DataFrame(data).join(to_dummy_var(data['Cuisine Style'].apply(to_list), new_columns=styles))

### Посмотрим распределение признака

In [144]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [145]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [146]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [147]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за этого мы имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [148]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [149]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [150]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [151]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

### загружаем чистые данные

In [152]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

### Вспомогательные функции

In [153]:
# строку в список
def to_list(data):
    pattern = re.compile(r'\[|\"|\'|\]')
    if isinstance(data, str):
        return re.sub(pattern, '', data).split(', ')
    else:
        return []


# приводим Prise Range к числовому формату
def transform_price_range(data):
    if data == '$':
        return 1
    elif data == '$$ - $$$':
        return 2
    else:
        return 0
    
def in_list_or_str(x, value):
    if isinstance(x, str):
        return value == x
    else:
        return value in x

# создать датафрейм, столбцы которого представлены уникальными значениями передаваемого столбца и
# содержат 1, если запись принадлежит к текущему столбцу, иначе 0
def to_dummy_var(column, new_columns=None):
    if not new_columns:
        new_columns = column.unique()
    store = []
    new_df = pd.DataFrame(columns=new_columns)
    for value in new_columns:
        series = column.apply(lambda x: 1 if in_list_or_str(x, value) else 0)
        new_df[f'Cuisine_Style_{value}'] = series
    return new_df

# количество каждой кухни
# функция используется для получения уникальных
def unique_cuisines(column):
    cuisines = set()
    for row in column:
        if row is not np.nan:
            cuisines = cuisines.union(set(row))
    return cuisines

In [154]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    value = df_output['Number of Reviews'].mean()
    df_output['Number of Reviews'].fillna(value, inplace=True)
    
    df_output['Price_Range_isNAN'] = pd.isna(df_output['Price Range']).astype('uint8')
    value = df_output['Price Range'].value_counts().index[0]
    df_output['Price Range'].fillna(value, inplace=True)
    
    df_output['Cuisine_Style_isNAN'] = pd.isna(df_output['Cuisine Style']).astype('uint8')
    df_output['Cuisine Style'].fillna("['Other']", inplace=True)
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    
    df_output['Price_Range_Numb'] = df_input['Price Range'].apply(transform_price_range)
    df_output = pd.get_dummies(df_output, columns=[ 'Price Range',])
    
    styles = list(unique_cuisines(df_output['Cuisine Style'].apply(to_list)))
    df_output = pd.DataFrame(df_output).join(to_dummy_var(df_output['Cuisine Style'].apply(to_list), new_columns=styles))

    # ################### 4. Feature Engineering ####################################################
    df_output['Count_Cuisine_Style'] = df_output['Cuisine Style'].apply(to_list).apply(lambda x: len(x))
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

#### Запускаем и проверяем что получилось

In [155]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [156]:
df_preproc.info()

In [157]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [158]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [159]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [160]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [161]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [162]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [163]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [164]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [171]:
test_data.sample(10)

In [172]:
test_data = test_data.drop(['Rating'], axis=1)

In [173]:
sample_submission

In [174]:
predict_submission = model.predict(test_data)

In [175]:
predict_submission

In [176]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)